In [4]:
from pathlib import Path
import pandas as pd

In [5]:
from tqdm import tqdm
import random
import pickle

In [6]:
import spacy
from spacy.tokens import Span
from spacy import displacy

In [154]:
# Define the colors for different labels
colors = {
    "SINTOMA": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
    "ENFERMEDAD": "linear-gradient(90deg, #ffcc00, #ff66cc)",
    "PROCEDIMIENTO": "linear-gradient(90deg, #00ccff, #0099cc)"
}

# Define the options with colors
options = {"ents": ["SINTOMA", "ENFERMEDAD", "PROCEDIMIENTO"], "colors": colors}


In [ ]:
# Load the annotations into a DataFrame
data = {
    'filename': ['doc1.txt', 'doc1.txt', 'doc1.txt', 'doc2.txt', 'doc2.txt'],
    'ann_id': ['T1', 'T2', 'T3', 'T1', 'T2'],
    'label': ['SINTOMA', 'SINTOMA', 'SINTOMA', 'SINTOMA', 'SINTOMA'],
    'start_span': [48, 100, 118, 20, 40],
    'end_span': [60, 115, 125, 35, 50],
    'text': ['obnubilación', 'dolor abdominal', 'vómitos', 'fiebre', 'mareo']
}

df = pd.DataFrame(data)


In [7]:
# Define the path to the folder containing the text files

clinical_notes_folder = '/home/alabarga/BSC/code/hackbcn-2024/data/clinical_notes/'

In [8]:
clinical_notes_path = Path(clinical_notes_folder)

In [9]:
# Create a dictionary to store the content of each file
texts = []

# Iterate over all text files in the folder and read their content
for text_file in clinical_notes_path.glob('*.txt'):
    with open(text_file, 'r', encoding='utf-8') as file:
        texts.append(
            {'filename': text_file.stem,
           'text': file.read()
            }
        )

In [10]:
clinical_notes = pd.DataFrame(texts)
clinical_notes

,filename,text
0,es-S0004-06142009000200013-1,"Mujer de 30 años, fumadora de unos seis cigarr..."
1,es-S0210-48062006000900012-2,Varón de 63 años. Consultó por hematuria macro...
2,es-S1130-05582016000100005-1,Presentamos el caso de un varón de 24 años ref...
3,es-S1139-76322009000700014-2,Paciente de 14 años que acude a la consulta po...
4,es-S1130-05582014000400008-1,Mujer de 63 años sin antecedentes de importanc...
...,...,...
995,es-S1130-14732008000500005-2,Mujer de 77 años de edad con único antecedente...
996,es-S0212-16112007000700016-1,"Varón de 32 años, sin antecedentes patológicos..."
997,es-S0210-48062005000600012-1,Mujer de 68 años con antecedentes de hipertens...
998,es-S1130-01082009000400009-1,"Varón de 50 años, diagnosticado de enfermedad ..."


In [11]:
clinical_notes.to_csv('../data/annotations/clinical_notes.csv', index=False)

In [ ]:
annotation_file_symptoms_train = '/home/alabarga/BSC/code/hackbcn-2024/data/symptemist-complete_240208/symptemist_train/subtask2-linking/symptemist_tsv_train_subtask2_complete+COMPOSITE.tsv'

In [102]:
annotation_file_symptoms_test = '/home/alabarga/BSC/code/hackbcn-2024/data/symptemist-complete_240208/symptemist_test/subtask2-linking/symptemist_tsv_test_subtask2+COMPOSITE.tsv'

In [104]:
annotation_file_diseases_train = '/home/alabarga/BSC/code/hackbcn-2024/data/distemist_zenodo/training/subtrack2_linking/distemist_subtrack2_training2_linking.tsv'

In [127]:
annotation_file_diseases_test = '/home/alabarga/BSC/code/hackbcn-2024/data/distemist_zenodo/test_annotated/subtrack2_linking/distemist_subtrack2_test_linking.tsv'

In [142]:
annotation_file_procedures_test ='/home/alabarga/BSC/code/hackbcn-2024/data/medprocner_gs_train+test+gazz+multilingual+crossmap_230808/medprocner_test/tsv/medprocner_tsv_test_subtask2.tsv'

In [141]:
annotation_file_procedures_train ='/home/alabarga/BSC/code/hackbcn-2024/data/medprocner_gs_train+test+gazz+multilingual+crossmap_230808/medprocner_train/tsv/medprocner_tsv_train_subtask2.tsv'

In [100]:
df_symtoms_train = pd.read_csv(annotation_file_symptoms_train, sep='\t')


In [110]:
df_symtoms_train = df_symtoms_train.get(['filename', 'label', 'span_ini', 'span_end', 'text', 'code'])
df_symtoms_train.columns = ['filename', 'label', 'span_start', 'span_end', 'span_text', 'code']

In [147]:
df_symtoms_train

,filename,label,span_start,span_end,span_text,code
0,es-S0365-66912011000600005-2,SINTOMA,333,361,«manchas» en el campo visual,246658005
1,es-S0004-06142010000300011-1,SINTOMA,649,716,5HIAA en orina de 24 horas estaba dentro de lo...,171250001
2,es-S1130-01082007000700011-2,SINTOMA,1463,1505,A nivel analítico no presentaba alteración,166315009
3,es-S0210-48062009000300017-1,SINTOMA,2713,2759,a nivel del cardias masa mamelonada y ulcerada,126825008
4,es-S1130-01082006000100014-1,SINTOMA,2282,2295,abdomen agudo,9209005
...,...,...,...,...,...,...
8975,es-S1134-80462009000800005-1,SINTOMA,745,750,dolor,22253000
8976,es-S1134-80462009000800005-1,SINTOMA,2587,2598,somnolencia,271782001
8977,es-S1134-80462009000800005-1,SINTOMA,2136,2143,vómitos,422400008
8978,es-S1134-80462009000800005-1,SINTOMA,2988,3006,sensación nauseosa,422587007


In [148]:
df_symtoms_test = pd.read_csv(annotation_file_symptoms_test, sep='\t')

In [149]:
df_symtoms_test = df_symtoms_test.get(['filename', 'label', 'span_ini', 'span_end', 'text', 'code'])
df_symtoms_test.columns = ['filename', 'label', 'span_start', 'span_end', 'span_text', 'code']

In [105]:
df_diseases_train = pd.read_csv(annotation_file_diseases_train, sep='\t')

In [111]:
df_diseases_train = df_diseases_train.get(['filename', 'label', 'off0', 'off1', 'span', 'code'])
df_diseases_train.columns = ['filename', 'label', 'span_start', 'span_end', 'span_text', 'code']

In [128]:
df_diseases_test = pd.read_csv(annotation_file_diseases_test, sep='\t')

In [129]:
df_diseases_test = df_diseases_test.get(['filename', 'label', 'off0', 'off1', 'span', 'code'])
df_diseases_test.columns = ['filename', 'label', 'span_start', 'span_end', 'span_text', 'code']

In [175]:
df_procedures_test = pd.read_csv(annotation_file_procedures_test, sep='\t')
df_procedures_test

,filename,label,start_span,end_span,text,code,sem_rel,is_abbrev,is_composite,need_context
0,S0004-06142006000100010-1,PROCEDIMIENTO,1298,1320,examen histopatológico,252416005,EXACT,False,False,False
1,S0004-06142006000100010-1,PROCEDIMIENTO,174,192,exploración física,5880005,EXACT,False,False,False
2,S0004-06142006000100010-1,PROCEDIMIENTO,216,237,ultrasonido abdominal,45036003,EXACT,False,False,False
3,S0004-06142006000100010-1,PROCEDIMIENTO,348,368,Urograma descendente,32265006,EXACT,False,False,False
4,S0004-06142006000100010-1,PROCEDIMIENTO,479,493,TAC helicoidal,116152004,EXACT,False,False,True
...,...,...,...,...,...,...,...,...,...,...
3613,es-S2340-98942015000100005-1,PROCEDIMIENTO,1732,1795,quimioterapia con trabectedina y adriamicina l...,367336001,NARROW,False,False,False
3614,es-S2340-98942015000100005-1,PROCEDIMIENTO,231,284,quimioterapia adyuvante con carboplatino y pac...,367336001,NARROW,False,False,False
3615,es-S2340-98942015000100005-1,PROCEDIMIENTO,465,468,TAC,77477000,EXACT,True,False,False
3616,es-S2340-98942015000100005-1,PROCEDIMIENTO,533,569,resección anterior de recto superior,172341002,EXACT,False,False,False


In [176]:
df_procedures_test = df_procedures_test.get(['filename', 'label', 'start_span', 'end_span', 'text', 'code'])
df_procedures_test.columns = ['filename', 'label', 'span_start', 'span_end', 'span_text', 'code']

In [177]:
df_procedures_train = pd.read_csv(annotation_file_procedures_train, sep='\t')
df_procedures_train

,filename,label,start_span,end_span,text,code,sem_rel,is_abbrev,is_composite,need_context
0,S0004-06142005000700014-1,PROCEDIMIENTO,1023,1044,Auscultación pulmonar,449264008,EXACT,False,False,False
1,S0004-06142005000700014-1,PROCEDIMIENTO,1143,1166,exploración neurológica,84728005,EXACT,False,False,False
2,S0004-06142005000700014-1,PROCEDIMIENTO,1307,1328,exploración urológica,302778005,EXACT,False,False,False
3,S0004-06142005000700014-1,PROCEDIMIENTO,1452,1461,palpación,113011001,EXACT,False,False,False
4,S0004-06142005000700014-1,PROCEDIMIENTO,1509,1525,transiluminación,103741002,EXACT,False,False,False
...,...,...,...,...,...,...,...,...,...,...
4852,es-S2254-28842014000300010-1,PROCEDIMIENTO,536,564,FAVI radiocefálica izquierda,439846009,EXACT,True,False,False
4853,es-S2254-28842014000300010-1,PROCEDIMIENTO,565,615,se le coloca un catéter temporal yugular izqui...,164780005,NARROW,False,False,False
4854,es-S2254-28842014000300010-1,PROCEDIMIENTO,728,758,FAVI húmero cefálica izquierda,27929005,NARROW,True,False,False
4855,es-S2254-28842014000300010-1,PROCEDIMIENTO,770,792,colocación del catéter,45211000,EXACT,False,False,False


In [178]:
df_procedures_train = df_procedures_train.get(['filename', 'label', 'start_span', 'end_span', 'text', 'code'])
df_procedures_train.columns = ['filename', 'label', 'span_start', 'span_end', 'span_text', 'code']

In [181]:
# Load spaCy model
nlp = spacy.blank("es")

def create_doc_with_annotations(text, annotations):
    doc = nlp(text)
    
    # Convert character-based annotations to token-based annotations
    token_annotations = []
    for start_char, end_char, label in annotations:
        start_token = None
        end_token = None
        for token in doc:
            if token.idx == start_char:
                start_token = token.i
            if token.idx + len(token) == end_char:
                end_token = token.i + 1
        
        if start_token is not None and end_token is not None:
            token_annotations.append((start_token, end_token, label))
        else:
            # Handle cases where tokenization doesn't align with annotation boundaries
            for token in doc:
                if token.idx >= start_char and token.idx + len(token) <= end_char:
                    if start_token is None:
                        start_token = token.i
                    end_token = token.i + 1
            token_annotations.append((start_token, end_token, label))

    # Create spans and add them to doc.ents
    spans = [Span(doc, start, end, label=label) for start, end, label in token_annotations]
    doc.ents = spans
    
    return doc


In [179]:
annotations = pd.concat([df_diseases_train, 
                         df_diseases_test, 
                         df_symtoms_test, 
                         df_symtoms_train,
                         df_procedures_train,
                         df_procedures_test])

In [186]:
annotations

,filename,label,span_start,span_end,span_text,code
0,es-S0210-56912007000900007-3,ENFERMEDAD,164,166,DM,73211009
1,es-S0210-56912007000900007-3,ENFERMEDAD,362,376,deshidratación,34095006
2,es-S0210-56912007000900007-3,ENFERMEDAD,575,590,hiperamilasemia,275739007
3,es-S0210-56912007000900007-3,ENFERMEDAD,715,733,pancreatitis aguda,197456007
4,es-S0210-56912007000900007-3,ENFERMEDAD,1402,1459,formación polipoidea sésil situada junto al es...,88580009
...,...,...,...,...,...,...
3613,es-S2340-98942015000100005-1,PROCEDIMIENTO,1732,1795,quimioterapia con trabectedina y adriamicina l...,367336001
3614,es-S2340-98942015000100005-1,PROCEDIMIENTO,231,284,quimioterapia adyuvante con carboplatino y pac...,367336001
3615,es-S2340-98942015000100005-1,PROCEDIMIENTO,465,468,TAC,77477000
3616,es-S2340-98942015000100005-1,PROCEDIMIENTO,533,569,resección anterior de recto superior,172341002


In [155]:
annotations['label'].value_counts()

label
SINTOMA          12014
PROCEDIMIENTO     8475
ENFERMEDAD        6222
Name: count, dtype: int64

In [182]:
# Process each unique filename
docs = []
error = 0
for i, row in tqdm(clinical_notes.iterrows()):
    filename = row['filename']
    text = row['text']
    annot = annotations[annotations['filename'] == filename][['span_start', 'span_end', 'label']].values.tolist()
    try:
        doc = create_doc_with_annotations(text, annot)
        docs.append({'filename':filename, 'doc':doc})
    except:
        errors =+ 1

print(f'{error} errors')        
    
    

1000it [00:07, 135.79it/s]

0 errors


In [183]:
doc = random.choice(docs).get('doc')
displacy.render(doc, style='ent', jupyter=True, options=options)

## Save results

In [184]:
with open('spacy_documents.pkl', 'wb') as f:
    pickle.dump(docs, f)

In [185]:
annotations.to_csv('annotations.csv', index=False)

In [1]:
texts

NameError: name 'texts' is not defined

In [ ]:
annotations[annotations['filename'] == filename]

In [ ]:
annot

In [ ]:
doc

In [72]:
nlp = spacy.blank("es")

In [85]:
len(text)

1791

In [83]:
doc = nlp(text)

In [84]:
len(doc)

290

In [79]:
[len(chunk) for chunk in text.split('\n')]

[210, 216, 641, 0, 157, 148, 0, 303, 106, 0, 0]

In [78]:
len(text)

1791

In [80]:
len(doc)

290

In [75]:
doc

Paciente masculino de 56 años de edad con antecedentes de serologia positiva para HIV y Hepatitis B que presenta cuadro de colecistitis aguda por la cual se le realiza colecistectomia laparoscopica de urgencia.
Durante el post-operatorio tardío, al mes de la cirugía, el paciente presenta un bilioma el cual es evacuado en forma percutánea. A su vez también se realiza una derivación biliar mediante colangiografia endoscopica.
Producto de la anterior el paciente presenta una pancreatitis aguda asociándose a ésta una colección peri-pancreática. Dicha colección se evacua mediante un drenaje percutáneo inicial, el cual prueba ser insuficiente presentando el paciente una importante colección retroperitoneal. Sin presentar mejoría, y con un débito persistente por el drenaje asociado a fiebre y deterioro del estado general, se realiza tomografía computada de abdomen y pelvis. Ésta informa líquido encapsulado con nivel hidroaéreo en el flanco derecho adyacente a la pared abdominal extendiéndose 

In [20]:
doc.ents

(serologia positiva para HIV y Hepatitis B,
 colección peri-pancreática,
 colección,
 colección retroperitoneal,
 débito persistente por el drenaje,
 fiebre,
 deterioro del estado general,
 líquido encapsulado con nivel hidroaéreo en el flanco derecho,
 lumboscopia, evacuando abundante material necrotico y purulento,
 registros febriles,
 cultivo positivo para Pseudomona Aureaginosa,
 afebril)

In [124]:
displacy.render(doc, style='ent', jupyter=True, options=options)